In [2]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

from sympy import *
import numpy as np


# NEWTON
-----------------

In [3]:
# Line search Algorithm

def linesearchUpdate(xk, deriv1_xk, Q):
    """
        xk:        Current value 
        deriv1_xk: First order derivative of function f(xk)
        Q :        Positive semidefinite matrix (when the variables/constraints are converted into the standard 2nd order form)
        
    """
    pk = -1 * deriv1_xk
    alpha = np.dot(-gradient_k,pk) / np.dot(np.dot(np.transpose(pk), Q), pk)
    print (alpha)
    xk1 = xk - alpha*pk
    return xk1
    
def newtonsUpdate(xk, deriv1, H):
    """
        xk:        Current value 
        deriv1 : 1st order derivative
        H      : Hessian matrix (2nd order derivative)
        
        pk (Search direction) = - inv(H).deriv1_xk 
    """
    pk = np.dot(np.linalg.inv(H),  deriv1)
    xk1 = xk - pk
    return xk1

In [13]:
x1 = Symbol('x1')
x2 = Symbol('x2')
x3 = Symbol('x3')
f = pow(x1,2) + pow(x2,2) + pow(x3,2)
deriv_f_x1 = f.diff(x1)
deriv_f_x2 = f.diff(x2)
deriv_f_x3 = f.diff(x3)
print (deriv_f_x1)
print (deriv_f_x2)
print (deriv_f_x3)

# When we convert the function f into the standard 2nd order form we ge the value of Q as
Q = np.array([[2,0,0],[0,2,0],[0,0,2]])
xk = np.array([1,1,1], dtype=float)
for i in range(2):
    deriv_f_x1_s = deriv_f_x1.subs(x1,xk[0]).subs(x2,xk[1]).subs(x3, xk[2])
    deriv_f_x2_s = deriv_f_x2.subs(x1,xk[0]).subs(x2,xk[1]).subs(x3, xk[2])
    deriv_f_x3_s = deriv_f_x2.subs(x1,xk[0]).subs(x2,xk[1]).subs(x3, xk[2])
#     print (deriv_f_x1_s, deriv_f_x2_s, deriv_f_x3_s)
    
    gradient_subs = np.array([deriv_f_x1_s, deriv_f_x2_s, deriv_f_x3_s])
#     gradient_norm = np.linalg.norm(gradient_subs)
    print ('')
    print (gradient_subs)
    xk1 = linesearchUpdate(xk=x_step, 
                                   deriv1_xk=gradient_subs, 
                                   Q=Q)
    print(x_step_next)
    xk = xk1
    

2*x1
2*x2
2*x3

[2.00000000000000 2.00000000000000 2.00000000000000]
2.00000000000000
[5.00000000000000 5.00000000000000 5.00000000000000]

[10.0000000000000 10.0000000000000 10.0000000000000]
0.400000000000000
[5.00000000000000 5.00000000000000 5.00000000000000]


In [ ]:
x1 = Symbol('x1')
x2 = Symbol('x2')
x3 = Symbol('x3')
f = pow(x1,2) + pow(x2,2) + pow(x3,2)
deriv_f_x1 = f.diff(x1)
deriv_f_x2 = f.diff(x2)
deriv_f_x3 = f.diff(x3)
print (deriv_f_x1)
print (deriv_f_x2)
print (deriv_f_x3)

# When we convert the function f into the standard 2nd order form we ge the value of Q as
Q = np.array([[2,0,0],[0,2,0],[0,0,2]])
xk = np.array([1,1,1])
c = np.array([0,0,0])
for i in range(4):
    gradient_k = np.dot(Q, xk) - c
    print (gradient_k)
    gradient_norm = np.linalg.norm(gradient_k)
    print ('')
    print (gradient_k, gradient_norm)
    x_step_next = linesearchUpdate(xk=x_step, 
                                   deriv1_xk=gradient_k, 
                                   Q=Q)
    print(x_step_next)
    xk = x_step_next
    

## Newtons Method:

#### Solution d(1)

In [4]:
x1 = Symbol('x1')
x2 = Symbol('x2')
x3 = Symbol('x3')
f = pow(x1,2) + pow(x2,2) + pow(x3,2)
deriv_f_x1 = f.diff(x1)
deriv_f_x2 = f.diff(x2)
deriv_f_x3 = f.diff(x3)

f = pow(x1,2) + pow(x2,2) + pow(x3,2)
deriv_1 = np.array([f.diff(x1), f.diff(x2), f.diff(x3)])
deriv_2 = np.array([[deriv_1[0].diff(x1), deriv_1[0].diff(x2), deriv_1[0].diff(x3)],
                    [deriv_1[1].diff(x1), deriv_1[1].diff(x2), deriv_1[1].diff(x3)],
                    [deriv_1[2].diff(x1), deriv_1[2].diff(x2), deriv_1[2].diff(x3)]])

print(deriv_1)


# H = np.array([[2,0,0],[0,2,0],[0,0,2]])
xk = np.array([0,0,0])

for i in range(5):
    gradient = np.array([eq.evalf(subs={x1:xk[0], x2:xk[1], x3:xk[2]}) for eq in deriv_1], dtype=float)
    hessian = np.array([cell.evalf(subs={x1:xk[0], x2:xk[1], x3:xk[2]}) 
                        for arrX in deriv_2 for cell in arrX], dtype=float).reshape(len(xk), len(xk))
    print ("####### Gradient Norm: ", np.linalg.norm(gradient))
    xk1 = newtonsUpdate(xk, deriv1=gradient, H=hessian)
    xk = xk1
    
    print(xk1)

[2*x1 2*x2 2*x3]
####### Gradient Norm:  0.0
[ 0.  0.  0.]
####### Gradient Norm:  0.0
[ 0.  0.  0.]
####### Gradient Norm:  0.0
[ 0.  0.  0.]
####### Gradient Norm:  0.0
[ 0.  0.  0.]
####### Gradient Norm:  0.0
[ 0.  0.  0.]


#### Solution d(2)

In [11]:
def newtons2Val(xk, is_print):
    gradient = np.array([eq.evalf(subs={x1:xk[0], x2:xk[1]}) for eq in deriv_1], dtype=float)
    if is_print:
        print(gradient)
        print('gradient NORM: ', np.linalg.norm(gradient))
    hessian = np.array([cell.evalf(subs={x1:xk[0], x2:xk[1]}) 
                        for arrX in deriv_2 for cell in arrX], dtype=float).reshape(len(xk), len(xk))

    xk1 = newtonsUpdate(xk, deriv1=gradient, H=hessian)
    return xk1

In [6]:
x1 = Symbol('x1')
x2 = Symbol('x2')
f = pow(x1,2) + 2*pow(x2,2) - 2*x1*x2 - 2*x2
deriv_1 = np.array([f.diff(x1), f.diff(x2)])
deriv_2 = np.array([[deriv_1[0].diff(x1),deriv_1[0].diff(x2)],
                    [deriv_1[1].diff(x1),deriv_1[1].diff(x2)]])
xk = np.array([0,0])

for i in range(5):
    xk = newtons2Val(xk, is_print=True)

[  1.18182126e-125  -2.00000000e+000]
gradient NORM:  2.0
[  4.72728505e-125   9.45457010e-125]
gradient NORM:  1.0570530726e-124
[  4.72728505e-125   9.45457010e-125]
gradient NORM:  1.0570530726e-124
[  4.72728505e-125   9.45457010e-125]
gradient NORM:  1.0570530726e-124
[  4.72728505e-125   9.45457010e-125]
gradient NORM:  1.0570530726e-124


#### Solution d(3)

In [7]:
x1 = Symbol('x1')
x2 = Symbol('x2')
f = 100*pow(x2-pow(x1,2),2) + pow(1-x1,2)
deriv_1 = np.array([f.diff(x1), f.diff(x2)])
deriv_2 = np.array([[deriv_1[0].diff(x1),deriv_1[0].diff(x2)],
                    [deriv_1[1].diff(x1),deriv_1[1].diff(x2)]])
print(deriv_1)
print(deriv_2)
xk = np.array([-1.2,1])

for i in range(5):
    xk = newtons2Val(xk, is_print=True)

[-400*x1*(-x1**2 + x2) + 2*x1 - 2 -200*x1**2 + 200*x2]
[[1200*x1**2 - 400*x2 + 2 -400*x1]
 [-400*x1 200]]
[-215.6  -88. ]
gradient NORM:  232.867687754
[-4.63781641 -0.12220679]
gradient NORM:  4.63942621407
[ 1146.45069037  -751.47563227]
gradient NORM:  1370.78984945
[ -4.73110379e-01  -1.98207786e-05]
gradient NORM:  0.473110379106
[ 22.38520499 -11.19265967]
gradient NORM:  25.0274455967


#### Solution d(4)

In [12]:
x1 = Symbol('x1')
x2 = Symbol('x2')
f = pow(x1+pow(x2, 2),4) + pow(x2,2)
deriv_1 = np.array([f.diff(x1), f.diff(x2)])
deriv_2 = np.array([[deriv_1[0].diff(x1),deriv_1[0].diff(x2)],
                    [deriv_1[1].diff(x1),deriv_1[1].diff(x2)]])
print(deriv_1)
print(deriv_2)
xk = np.array([2,-2])

for i in range(25):
    if (i+1)%5 == 0:
        xk = newtons2Val(xk, is_print=True)
    else:
        xk = newtons2Val(xk, is_print=False)


[4*(x1 + x2**2)**3 8*x2*(x1 + x2**2)**3 + 2*x2]
[[12*(x1 + x2**2)**2 24*x2*(x1 + x2**2)**2]
 [24*x2*(x1 + x2**2)**2 48*x2**2*(x1 + x2**2)**2 + 8*(x1 + x2**2)**3 + 2]]
[  6.78567249 -29.28120165]
gradient NORM:  30.0571808614
[ 0.3465133  -0.49853495]
gradient NORM:  0.607131423051
[  9.78697590e-04  -1.38103693e-08]
gradient NORM:  0.000978697589731
[  2.23501084e-06  -6.45184384e-29]
gradient NORM:  2.23501083512e-06
[  5.10400096e-09  -1.87647116e-62]
gradient NORM:  5.10400095597e-09


#### Solution d(5) :

In [15]:
c = 1
x1 = Symbol('x1')
x2 = Symbol('x2')
f = pow(x1-1, 2) + pow(x2-1, 2) + c*pow((pow(x1,0) + pow(x2,2) - 0.25), 2)
deriv_1 = np.array([f.diff(x1), f.diff(x2)])
deriv_2 = np.array([[deriv_1[0].diff(x1),deriv_1[0].diff(x2)],
                    [deriv_1[1].diff(x1),deriv_1[1].diff(x2)]])
print(deriv_1)
print(deriv_2)
xk = np.array([2,-2])

for i in range(5):
#     if (i+1)%5 == 0:
    xk = newtons2Val(xk, is_print=True)
#     else:
#         xk = newtons2Val(xk, is_print=False)

[2*x1 - 2 4*x2*(x2**2 + 0.75) + 2*x2 - 2]
[[2 0]
 [0 12*x2**2 + 5.0]]
[  2. -44.]
gradient NORM:  44.0454310911
[  4.72728505e-125  -1.42524097e+001]
gradient NORM:  14.2524097073
[  4.72728505e-125  -5.03595579e+000]
gradient NORM:  5.035955789
[  4.72728505e-125  -1.38897465e+000]
gradient NORM:  1.38897464944
[  4.72728505e-125   1.81756576e-001]
gradient NORM:  0.181756576395


In [18]:
c = 10
x1 = Symbol('x1')
x2 = Symbol('x2')
f = pow(x1-1, 2) + pow(x2-1, 2) + c*pow((pow(x1,0) + pow(x2,2) - 0.25), 2)
deriv_1 = np.array([f.diff(x1), f.diff(x2)])
deriv_2 = np.array([[deriv_1[0].diff(x1),deriv_1[0].diff(x2)],
                    [deriv_1[1].diff(x1),deriv_1[1].diff(x2)]])
print(deriv_1)
print(deriv_2)
xk = np.array([2,-2])

for i in range(20):
    if (i+1)%5 == 0:
        xk = newtons2Val(xk, is_print=True)
    else:
        xk = newtons2Val(xk, is_print=False)

[2*x1 - 2 40*x2*(x2**2 + 0.75) + 2*x2 - 2]
[[2 0]
 [0 120*x2**2 + 32.0]]
[  4.72728505e-125  -1.81199080e+000]
gradient NORM:  1.81199080103
[  4.72728505e-125  -8.04495554e-017]
gradient NORM:  8.0449555424e-17
[  4.72728505e-125  -8.04495554e-017]
gradient NORM:  8.0449555424e-17
[  4.72728505e-125  -8.04495554e-017]
gradient NORM:  8.0449555424e-17


In [20]:
c = 100
x1 = Symbol('x1')
x2 = Symbol('x2')
f = pow(x1-1, 2) + pow(x2-1, 2) + c*pow((pow(x1,0) + pow(x2,2) - 0.25), 2)
deriv_1 = np.array([f.diff(x1), f.diff(x2)])
deriv_2 = np.array([[deriv_1[0].diff(x1),deriv_1[0].diff(x2)],
                    [deriv_1[1].diff(x1),deriv_1[1].diff(x2)]])
print(deriv_1)
print(deriv_2)
xk = np.array([2,-2])

for i in range(20):
    if (i+1)%5 == 0:
        xk = newtons2Val(xk, is_print=True)
    else:
        xk = newtons2Val(xk, is_print=False)

[2*x1 - 2 400*x2*(x2**2 + 0.75) + 2*x2 - 2]
[[2 0]
 [0 1200*x2**2 + 302.0]]
[  4.72728505e-125  -1.94004158e+001]
gradient NORM:  19.4004158389
[  4.72728505e-125   9.37571443e-018]
gradient NORM:  9.37571443109e-18
[  4.72728505e-125   9.37571443e-018]
gradient NORM:  9.37571443109e-18
[  4.72728505e-125   9.37571443e-018]
gradient NORM:  9.37571443109e-18


# BFGS UPDATE
---------

In [33]:
def BFGSUpdate(xk, Bk, deriv1_xk, Q):
    """
        xk:        Current value 
        Bk:        Approximated 2nd order matrix
        deriv1_xk: First order derivative of function f(xk)
        Q :        Q formed when we convert the function in standard form
                   Also equivallent to Hessian matrix, Computation is needed only once
                   No update is required
        
        pk (Search direction) = - inv(Bk).deriv1_xk 
    """
    pk = -1 * np.dot(np.linalg.inv(Bk),deriv1_xk)
    alpha = np.dot(np.transpose(pk), deriv1_xk) / np.dot(np.dot(np.transpose(pk), Q), pk)
    xk1 = xk - alpha*pk
    return xk1

def updateBk(step, deriv_1, xk, xk1, gradient_xk, Bk):
    """
        step:             Step Num, int
        deriv_1           1st order derivative of the input function 
        xk:               previous solution
        xk1:              New Solution
        gradient_xk:      deriv_1 evaluated at xk1
        Bk:               Approximated 2nd order matrix
    """
    gradient_xk1 =  np.array([eq.evalf(subs={x1:xk1[0], x2:xk1[1], x3:xk1[2]}) 
                              for eq in deriv_1], dtype=float)
    print ('Gradient at xk%s: '%str(step), gradient_xk1)
    Sk = xk1 - xk
    Yk = gradient_xk1 - gradient_xk
    numerator = np.dot(np.dot(Bk,Sk),np.transpose(np.dot(Bk,Sk)))
    denominator = np.dot(np.transpose(Sk),np.dot(Bk,Sk))
    Bk1 = Bk - (numerator/denominator) + (np.dot(Yk, np.transpose(Yk))/np.dot(np.transpose(Yk), Yk))
    
    return Bk1

#### Solution d(1)

In [34]:
f = pow(x1,2) + pow(x2,2) + pow(x3,2)
deriv_f_x1 = f.diff(x1)
deriv_f_x2 = f.diff(x2)
deriv_f_x3 = f.diff(x3)

f = pow(x1,2) + pow(x2,2) + pow(x3,2)
deriv_1 = np.array([f.diff(x1), f.diff(x2), f.diff(x3)])
print ('First order Derivative: ', deriv_1)
Q = np.array([[2,0,0],[0,2,0],[0,0,2]])  # Q is obtained by converting the funciton into quadratic form, which is equivallent to 2nd order derivative in all our case

# Initialize BK
Bk = np.eye(3)

# Initialize Xk
xk = np.array([0,0,0])

for step in range(5):
    gradient = np.array([eq.evalf(subs={x1:xk[0], x2:xk[1], x3:xk[2]}) for eq in deriv_1], dtype=float)
    print ('Gradient at xk%s is: '%str(step), gradient)
    gradientNorm = np.linalg.norm(gradient)
    print ("####### Gradient Norm: ", gradientNorm)
    if gradientNorm == 0:
        break
        
    xk1 = BFGSUpdate(xk=xk, Bk=Bk, deriv1_xk=gradient, Q=Q)
    Bk1 = updateBk(step, deriv_1, xk, xk1, gradient_xk, Bk)
    
    # Parameter UPdate
    xk = xk1
    Bk = Bk1
    
    print(xk1)

First order Derivative:  [2*x1 2*x2 2*x3]
Gradient at xk0 is:  [ 0.  0.  0.]
####### Gradient Norm:  0.0


#### Solution d(2)

In [ ]:
f = pow(x1,2) + pow(x2,2) + pow(x3,2)
deriv_f_x1 = f.diff(x1)
deriv_f_x2 = f.diff(x2)
deriv_f_x3 = f.diff(x3)

f = pow(x1,2) + pow(x2,2) + pow(x3,2)
deriv_1 = np.array([f.diff(x1), f.diff(x2), f.diff(x3)])
print ('First order Derivative: ', deriv_1)
Q = np.array([[2,0,0],[0,2,0],[0,0,2]])  # Q is obtained by converting the funciton into quadratic form, which is equivallent to 2nd order derivative in all our case

# Initialize BK
Bk = np.eye(3)

# Initialize Xk
xk = np.array([0,0,0])

for step in range(5):
    gradient = np.array([eq.evalf(subs={x1:xk[0], x2:xk[1], x3:xk[2]}) for eq in deriv_1], dtype=float)
    print ('Gradient at xk%s is: '%str(step), gradient)
    gradientNorm = np.linalg.norm(gradient)
    print ("####### Gradient Norm: ", gradientNorm)
    if gradientNorm == 0:
        break
        
    xk1 = BFGSUpdate(xk=xk, Bk=Bk, deriv1_xk=gradient, Q=Q)
    Bk1 = updateBk(step, deriv_1, xk, xk1, gradient_xk, Bk)
    
    # Parameter UPdate
    xk = xk1
    Bk = Bk1
    
    print(xk1)